In [52]:
!pip install pandas
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd
import sys
import matplotlib 
import numpy as np

%matplotlib inline

In [53]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Reg

In [0]:
import pandas as pd 
import pprint as pp
from datetime import datetime
date_format = "%Y-%m-%d"
import time
from multiprocessing.dummy import Pool as ThreadPool 

# Convert MIN string into int
def ConvertMinutes(x):
    return float(x.split(':')[0])

# Convert GAME_DATE_EST into datetime 
def ConvertGameDate(x):
    return datetime.strptime(x.split('T')[0], date_format)

# Calculate Offensive Rating
def CalcOffRtg(x):
    poss = 0.96*((x['FGA'])+(x['TO'])+0.44*(x['FTA'])-(x['OREB']))
    off_rtg = ((x['PTS'])/poss) * 100
    return off_rtg
  
def ImportData(team_stats_csv, summaries_csv):
    team_stats = pd.read_csv(team_stats_csv)
    summaries = pd.read_csv(summaries_csv)
    game_dates = summaries[['GAME_DATE_EST','HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'GAME_ID']].copy()

    team_stats = pd.merge(team_stats, game_dates, on=['GAME_ID'])
    team_stats = team_stats.drop(columns=['TEAM_ABBREVIATION', 'TEAM_CITY'])

    # Change GAME_DATE_EST to datetime object
    team_stats['GAME_DATE_EST'] = team_stats['GAME_DATE_EST'].apply(ConvertGameDate)


    # Calculate Off Rtg
    team_stats['OFF_RTG'] = team_stats.apply(CalcOffRtg, axis=1)

    # Add Home or Away suffixes
    away, home = [x for _, x in team_stats.groupby(team_stats['TEAM_ID'] == team_stats['HOME_TEAM_ID'])]
    home = home.add_suffix('_HOME')
    away = away.add_suffix('_AWAY')

    team_stats = pd.merge(home, away, left_on=['GAME_ID_HOME'], right_on=['GAME_ID_AWAY'])
    team_stats = team_stats.drop(columns=['Unnamed: 0_HOME', 'Unnamed: 0_AWAY'])
    team_stats['MIN'] = team_stats['MIN_HOME'].apply(ConvertMinutes)
    team_stats = team_stats.drop(columns=['MIN_HOME', 'MIN_AWAY'])
    
    # Add binary WIN_HOME field
    team_stats['WIN_HOME'] = 0
    team_stats.loc[team_stats['PTS_HOME'] > team_stats['PTS_AWAY'], 'WIN_HOME'] = 1
    return team_stats

In [0]:
# Main
df_2018 = ImportData(r'/content/gdrive/My Drive/ML Project Fall 2018/Code/2017-2018_team_stats.csv', r'/content/gdrive/My Drive/ML Project Fall 2018/Code/2017-2018_summary.csv')
df_2017 = ImportData(r'/content/gdrive/My Drive/ML Project Fall 2018/Code/2016-2017_team_stats.csv', r'/content/gdrive/My Drive/ML Project Fall 2018/Code/2016-2017_summary.csv')
df_2016 = ImportData(r'/content/gdrive/My Drive/ML Project Fall 2018/Code/2015-2016_team_stats.csv', r'/content/gdrive/My Drive/ML Project Fall 2018/Code/2015-2016_summary.csv')
df_2015 = ImportData(r'/content/gdrive/My Drive/ML Project Fall 2018/Code/2014-2015_team_stats.csv', r'/content/gdrive/My Drive/ML Project Fall 2018/Code/2014-2015_summary.csv')

df = pd.DataFrame()
df = df.append(df_2018)
df = df.append(df_2017)
df = df.append(df_2016)
df = df.append(df_2015)


In [0]:
df1 = pd.DataFrame()
df1 = pd.concat([df1,df[['GAME_ID_HOME','PTS_AWAY','PTS_HOME']]], axis=1)

In [0]:
#df2 = pd.DataFrame()
#df2['GAME_ID_HOME'] = df1['GAME_ID_HOME']
#df1 = df1.drop(['GAME_ID_HOME'], axis=1)
#df_norm = (df1 - df1.mean()) / (df1.max() - df1.min())
#df1 = pd.concat([df2,df_norm], axis=1)

In [57]:
Location = r'/content/gdrive/My Drive/ML Project Fall 2018/Code/final_with_ranks.csv'
team_stats = pd.read_csv(Location)
team_stats

,Unnamed: 0,CURRENT_GAME_ID,GAME_DATE_EST,TEAM_NAME_AWAY,TEAM_NAME_HOME,WIN_HOME,NET_RTG_HOME,NET_RTG_AWAY,WIN_PERCENTAGE_A,WIN_PERCENTAGE_B,DAYS_SINCE_LAST_GAME_A,DAYS_SINCE_LAST_GAME_B,GAME_ID_HOME,EXPERT_RANK_HOME,EXPERT_RANK_AWAY
0,0,21701213,2018-04-10,Hornets,Pacers,0,-0.071810,-0.130760,0.168894,-0.035384,-0.009788,0.000677,21701213,0.357431,0.056420
1,1,21701214,2018-04-10,76ers,Hawks,0,0.026551,0.138002,-0.231106,0.431282,-0.009788,-0.009424,21701214,0.460879,0.159868
2,2,21701216,2018-04-10,Suns,Mavericks,0,0.043764,-0.015764,-0.297772,-0.435384,-0.009788,-0.009424,21701216,0.116051,0.332282
3,3,21701217,2018-04-10,Warriors,Jazz,1,0.307173,0.270377,0.302228,-0.035384,-0.009788,-0.009424,21701217,-0.125328,-0.495304
4,4,21701218,2018-04-10,Rockets,Lakers,0,0.041621,-0.164731,-0.164439,0.364616,-0.009788,-0.004374,21701218,0.219500,-0.426339
5,5,21701215,2018-04-10,Celtics,Wizards,1,-0.082503,-0.016749,-0.164439,0.097949,0.000262,-0.009424,21701215,-0.297742,-0.322891
6,6,21701207,2018-04-09,Cavaliers,Knicks,0,-0.184613,-0.041540,-0.231106,0.231282,-0.009788,-0.004374,21701207,0.391913,-0.460822
7,7,21701204,2018-04-09,Raptors,Pistons,0,0.109514,0.067382,0.102228,0.164616,-0.014813,-0.014475,21701204,0.185017,-0.184960
8,8,21701205,2018-04-09,Bulls,Nets,1,0.010085,-0.188161,-0.031106,-0.168718,-0.009788,-0.004374,21701205,0.426396,0.504696
9,9,21701206,2018-04-09,Thunder,Heat,0,0.069137,0.026314,0.102228,0.097949,-0.004763,-0.009424,21701206,0.081569,-0.391856


In [0]:
team_stats = pd.merge(team_stats, df1, left_on=['CURRENT_GAME_ID'], right_on=['GAME_ID_HOME'], sort=False)


In [0]:
#team_stats1 = team_stats
temp = pd.get_dummies(team_stats[['TEAM_NAME_AWAY']])
team_stats = team_stats.drop(columns=['TEAM_NAME_AWAY'])
team_stats = pd.concat([team_stats, temp], axis=1)

temp = pd.get_dummies(team_stats[['TEAM_NAME_HOME']])
team_stats = team_stats.drop(columns=['TEAM_NAME_HOME'])
team_stats = pd.concat([team_stats, temp], axis=1)


In [0]:
team_stats = team_stats.drop(['CURRENT_GAME_ID','Unnamed: 0','GAME_ID_HOME_x','GAME_ID_HOME_y'],axis=1)
#team_stats = team_stats.drop(['GAME_DATE_EST'],axis=1)


In [0]:
team_stats = team_stats.drop(['TEAM_NAME_AWAY','TEAM_NAME_HOME'],axis=1)

In [0]:
team_stats = team_stats.drop_duplicates()
team_stats1 = team_stats

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
X=team_stats.drop(['WIN_HOME','PTS_AWAY','PTS_HOME'],axis=1)
#y=team_stats[[i for i in list(team_stats.columns) if (i == 'WIN_HOME_A0'or i=='WIN_AWAY_A0'or i=='PLUS_MINUS_HOME_A0'or i=='PLUS_MINUS_AWAY_A0')]]
y=team_stats[[i for i in list(team_stats.columns) if (i == 'PTS_AWAY' or i=='PTS_HOME' or i=='WIN_HOME')]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [0]:
test_end_index = int(team_stats1.index[team_stats1['GAME_DATE_EST'] == "2017-12-25"][-1])
test_start_index = int(team_stats1.index[team_stats1['GAME_DATE_EST'] == "2018-04-10"][0])
train_end_index = int(team_stats1.index[team_stats1['GAME_DATE_EST'] == "2017-01-25"][-1])
train_start_index = int(team_stats1.index[team_stats1['GAME_DATE_EST'] == "2017-12-25"][0])

X_train, X_test = X.iloc[train_start_index:train_end_index], X.iloc[test_start_index:test_end_index]
y_train, y_test = y.iloc[train_start_index:train_end_index], y.iloc[test_start_index:test_end_index]

In [0]:
y_train = y_train.drop(['WIN_HOME'], axis =1)
y_win = pd.DataFrame()
y_win['WIN_HOME'] = y_test['WIN_HOME']
y_test = y_test.drop(['WIN_HOME'], axis=1)

In [0]:
X_train = np.array(X_train)

y_train = np.array(y_train)
X_train.shape

(4026, 68)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
# from keras.utils import to_categorical
from keras.layers import Embedding
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(20, activation='relu'))
model.add(Dense(2, activation='linear'))
model.compile(loss='logcosh', optimizer='Adam', metrics=['mae'], lr=0.01)
model.fit(X_train, y_train, batch_size=500, epochs=500)
print(model.summary())
model.save('/content/gdrive/My Drive/ML Project Fall 2018/trained_model.h5')

Epoch 1/500
4026/4026 [==============================] - 2s 526us/step - loss: 101.6647 - mean_absolute_error: 102.3579
Epoch 2/500
4026/4026 [==============================] - 1s 143us/step - loss: 87.5245 - mean_absolute_error: 88.2176
Epoch 3/500
4026/4026 [==============================] - 1s 145us/step - loss: 30.2439 - mean_absolute_error: 30.9300
Epoch 4/500
4026/4026 [==============================] - 1s 148us/step - loss: 17.8586 - mean_absolute_error: 18.5401
Epoch 5/500
4026/4026 [==============================] - 1s 147us/step - loss: 13.7873 - mean_absolute_error: 14.4609
Epoch 6/500
4026/4026 [==============================] - 1s 144us/step - loss: 11.9450 - mean_absolute_error: 12.6192
Epoch 7/500
4026/4026 [==============================] - 1s 144us/step - loss: 10.9790 - mean_absolute_error: 11.6496
Epoch 8/500
4026/4026 [==============================] - 1s 146us/step - loss: 10.6262 - mean_absolute_error: 11.2974
Epoch 9/500
4026/4026 [==============================]

In [0]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

# R1

In [0]:
from keras import losses
accuracy=0.0
mae=0.0
win=[]
y_win = np.array(y_win)
X_test = np.array(X_test)
y_test = np.array(y_test)
test_preds = model.predict(X_test)
for i in test_preds:
  if i[0]>i[1]:
    win.append(0)
  else:
    win.append(1)
win = np.array(win)
print("R1 Test Accuracy: " + str(accuracy_score(y_win, win))) 

R1 Test Accuracy: 0.6573248407643312


mean_squared_logarithmic_error - 0.6424116424116424
mean_squared_error - 0.636
mean_absolute_error - 0.624
mean_absolute_percentage_error - 0.602
logcosh - 0.6237006237006237

# R2

In [0]:
from keras import losses
accuracy=0.0
mae=0.0
win=[]
y_win = np.array(y_win)
X_test = np.array(X_test)
y_test = np.array(y_test)
test_preds = model.predict(X_test)
for i in test_preds:
  if i[0]>i[1]:
    win.append(0)
  else:
    win.append(1)
win = np.array(win)
print("R2 Test Accuracy: " + str(accuracy_score(y_win, win))) 

R2 Test Accuracy: 0.5949044585987261


In [0]:
X_test.shape

(785, 68)

mean_squared_logarithmic_error - 0.6573248407643312
mean_squared_error - 0.5885350318471337
mean_absolute_error -  0.6101910828025477
mean_absolute_percentage_error - 0.5885350318471337
logcosh - 0.5949044585987261

# R3

In [0]:
# Full Season
#date_list = list(df["GAME_DATE_EST"][0:1210].drop_duplicates())

# Fraction of Season
date_list = list(team_stats1["GAME_DATE_EST"][0:200].drop_duplicates())

train_end_val = "2016-10-28"


In [64]:
from tqdm import tqdm
# Predicting one day at a time - NN
accuracies = []
games_per_day = []
total_games = 0
full_season_acc=[]

for date in tqdm(date_list):
    test_end = int(team_stats1.index[team_stats1['GAME_DATE_EST'] == date][-1])
    test_start = int(team_stats1.index[team_stats1['GAME_DATE_EST'] == date][0])
    train_end = int(team_stats1.index[team_stats1['GAME_DATE_EST'] == train_end_val][-1])
    train_start = int(team_stats1.index[team_stats1['GAME_DATE_EST'] == date][-1] + 1)
    
    games = test_end - test_start
    total_games += games
    games_per_day.append(games)
    
    temp = team_stats1['GAME_DATE_EST']
    team_stats1 = team_stats1.drop(['GAME_DATE_EST'],axis=1)
    
    X=team_stats1.drop(['WIN_HOME','PTS_AWAY','PTS_HOME'],axis=1)
    #y=team_stats[[i for i in list(team_stats.columns) if (i == 'WIN_HOME_A0'or i=='WIN_AWAY_A0'or i=='PLUS_MINUS_HOME_A0'or i=='PLUS_MINUS_AWAY_A0')]]
    y=team_stats1[[i for i in list(team_stats1.columns) if (i == 'PTS_AWAY' or i=='PTS_HOME' or i=='WIN_HOME')]]

    #x = df.drop(columns=['WIN_HOME'])
    
    if test_start == test_end:
        X_train, X_test = X.iloc[train_start:train_end], X.iloc[[test_start]]
        y_train, y_test = y.iloc[train_start:train_end], y.iloc[[test_start]]
    elif train_start == train_end:
        X_train, X_test = X.iloc[[train_start]], X.iloc[test_start:test_end]
        y_train, y_test = y.iloc[[train_start]], y.iloc[test_start:test_end]
    else:
        X_train, X_test = X.iloc[train_start:train_end], X.iloc[test_start:test_end]
        y_train, y_test = y.iloc[train_start:train_end], y.iloc[test_start:test_end]
        
    y_train = y_train.drop(['WIN_HOME'], axis =1)
    y_win = pd.DataFrame()
    y_win['WIN_HOME'] = y_test['WIN_HOME']
    y_test = y_test.drop(['WIN_HOME'], axis=1)
    
    X_train = np.array(X_train)

    y_train = np.array(y_train)
    print(X_train.shape)
        
    model = Sequential()
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(250, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(2, activation='linear'))
    model.compile(loss='mean_squared_logarithmic_error', optimizer='Adam', metrics=['mae'], lr=0.01)
    model.fit(X_train, y_train, batch_size=500, epochs=500)
    
    
    win=[]
    y_win = np.array(y_win)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    test_preds = model.predict(X_test)
    for i in test_preds:
      if i[0]>i[1]:
        win.append(0)
      else:
        win.append(1)
    win = np.array(win)
    accuracy = accuracy_score(y_win, win)
    accuracies.append(accuracy * games)
        
    team_stats1["GAME_DATE_EST"] = temp

print("\nResults for NN model:") 
print(np.sum(accuracies)/total_games)
full_season_acc.append(np.sum(accuracies)/total_games)




  0%|          | 0/26 [00:00<?, ?it/s]

(4801, 68)
Epoch 1/500
4801/4801 [==============================] - 2s 517us/step - loss: 14.3522 - mean_absolute_error: 101.4234
Epoch 2/500
4801/4801 [==============================] - 1s 142us/step - loss: 1.7850 - mean_absolute_error: 65.8403
Epoch 3/500
4801/4801 [==============================] - 1s 140us/step - loss: 0.1618 - mean_absolute_error: 43.3220
Epoch 4/500
4801/4801 [==============================] - 1s 139us/step - loss: 0.3453 - mean_absolute_error: 79.0999
Epoch 5/500
4801/4801 [==============================] - 1s 140us/step - loss: 0.1247 - mean_absolute_error: 38.4000
Epoch 6/500
4801/4801 [==============================] - 1s 143us/step - loss: 0.0285 - mean_absolute_error: 13.7591
Epoch 7/500
4801/4801 [==============================] - 1s 142us/step - loss: 0.0354 - mean_absolute_error: 14.8158
Epoch 8/500
4801/4801 [==============================] - 1s 143us/step - loss: 0.0266 - mean_absolute_error: 12.9883
Epoch 9/500
4801/4801 [============================



  4%|▍         | 1/26 [05:40<2:21:56, 340.65s/it]

(4792, 68)
Epoch 1/500
4792/4792 [==============================] - 3s 525us/step - loss: 14.6702 - mean_absolute_error: 101.6216
Epoch 2/500
4792/4792 [==============================] - 1s 141us/step - loss: 1.6857 - mean_absolute_error: 65.2964
Epoch 3/500
4792/4792 [==============================] - 1s 140us/step - loss: 0.1483 - mean_absolute_error: 41.8029
Epoch 4/500
4792/4792 [==============================] - 1s 144us/step - loss: 0.3470 - mean_absolute_error: 80.1197
Epoch 5/500
4792/4792 [==============================] - 1s 142us/step - loss: 0.1302 - mean_absolute_error: 39.9701
Epoch 6/500
4792/4792 [==============================] - 1s 140us/step - loss: 0.0247 - mean_absolute_error: 13.1285
Epoch 7/500
4792/4792 [==============================] - 1s 141us/step - loss: 0.0301 - mean_absolute_error: 14.0691
Epoch 8/500
4792/4792 [==============================] - 1s 140us/step - loss: 0.0243 - mean_absolute_error: 12.7301
Epoch 9/500
4792/4792 [============================



  8%|▊         | 2/26 [11:22<2:16:22, 340.94s/it]

...



 19%|█▉        | 5/26 [28:43<2:00:24, 344.02s/it]

(4763, 68)
Epoch 1/500
4763/4763 [==============================] - 3s 648us/step - loss: 15.9928 - mean_absolute_error: 102.3141
Epoch 2/500
4763/4763 [==============================] - 1s 147us/step - loss: 2.8701 - mean_absolute_error: 77.5486
Epoch 3/500
4763/4763 [==============================] - 1s 145us/step - loss: 0.1205 - mean_absolute_error: 31.5005
Epoch 4/500
4763/4763 [==============================] - 1s 144us/step - loss: 0.3072 - mean_absolute_error: 73.1162
Epoch 5/500
4763/4763 [==============================] - 1s 145us/step - loss: 0.1678 - mean_absolute_error: 47.4199
Epoch 6/500
4763/4763 [==============================] - 1s 144us/step - loss: 0.0456 - mean_absolute_error: 18.5890
Epoch 7/500
4763/4763 [==============================] - 1s 145us/step - loss: 0.0330 - mean_absolute_error: 14.2545
Epoch 8/500
4763/4763 [==============================] - 1s 146us/step - loss: 0.0265 - mean_absolute_error: 13.0189
Epoch 9/500
4763/4763 [============================



 23%|██▎       | 6/26 [34:30<1:54:57, 344.86s/it]

(4757, 68)
Epoch 1/500
4757/4757 [==============================] - 3s 631us/step - loss: 14.2109 - mean_absolute_error: 101.4064
Epoch 2/500
4757/4757 [==============================] - 1s 144us/step - loss: 1.7554 - mean_absolute_error: 67.2657
Epoch 3/500
4757/4757 [==============================] - 1s 144us/step - loss: 0.1321 - mean_absolute_error: 37.8746
Epoch 4/500
4757/4757 [==============================] - 1s 143us/step - loss: 0.3260 - mean_absolute_error: 76.2823
Epoch 5/500
4757/4757 [==============================] - 1s 144us/step - loss: 0.1303 - mean_absolute_error: 39.6691
Epoch 6/500
4757/4757 [==============================] - 1s 145us/step - loss: 0.0257 - mean_absolute_error: 13.3820
Epoch 7/500
4757/4757 [==============================] - 1s 144us/step - loss: 0.0301 - mean_absolute_error: 14.0944
Epoch 8/500
4757/4757 [==============================] - 1s 144us/step - loss: 0.0251 - mean_absolute_error: 12.8294
Epoch 9/500
4757/4757 [============================

KeyboardInterrupt: ignored

10 games - Results for NN model:
0.6923076923076923

